In [1]:
import cx_Oracle
oracle_dsn = cx_Oracle.makedsn(host="localhost", port=1521, sid="xe")
global conn
conn = cx_Oracle.connect("scott", "tiger", dsn=oracle_dsn)
cursor = conn.cursor()
cursor.execute("""
    CREATE TABLE MEMBERS(
        NAME VARCHAR2(20),
        PHONE VARCHAR2(20),
        EMAIL VARCHAR2(30),
        AGE NUMBER(3),
        GRADE NUMBER(2),
        ETC VARCHAR2(50)
    )
""")

In [10]:
def main():
    cursor = conn.cursor()
    while True:
        print("1:입력","2:전체출력","3:삭제", "4:이름찾기","5:내보내기(CSV)", 
              "9:종료", sep="|", end="")
        menu = int(input("메뉴 선택 : "))
        if menu == 1:
            insert_customer_info(cursor)
        elif menu == 2:
            print_customers(cursor)
        elif menu == 3:
            delete_customer(cursor)
        elif menu == 4:
            search_customer(cursor)
        elif menu == 5:
            save_customer_csv(cursor)
        elif menu == 9:
            conn.close()
            break
if __name__=='__main__':
    import cx_Oracle
    oracle_dsn = cx_Oracle.makedsn(host="localhost", port=1521, sid="xe")
    global conn
    conn = cx_Oracle.connect("scott", "tiger", dsn=oracle_dsn)
    main()

1:입력|2:전체출력|3:삭제|4:이름찾기|5:내보내기(CSV)|9:종료메뉴 선택 : 1
이름: 신정우
전화번호: 010-3434-2131
이메일: gsl@qw.com
나이 : 32
고객등급(1~5) : 2
세부사항: n
1:입력|2:전체출력|3:삭제|4:이름찾기|5:내보내기(CSV)|9:종료메뉴 선택 : 2
  NAME          PHONE       EMAIL  AGE  GRADE ETC
0  강기영          12121  qwq@qw.com   23      3   n
1  신정우  010-3434-2131  gsl@qw.com   32      2   n
1:입력|2:전체출력|3:삭제|4:이름찾기|5:내보내기(CSV)|9:종료메뉴 선택 : 1
이름: 신정우
전화번호: 010-2323-3231
이메일: glwe@qw.com
나이 : 23
고객등급(1~5) : 1
세부사항: n
1:입력|2:전체출력|3:삭제|4:이름찾기|5:내보내기(CSV)|9:종료메뉴 선택 : 4
검색할 이름은? 신정우
('신정우', '010-3434-2131', 'gsl@qw.com', 32, 2, 'n')
('신정우', '010-2323-3231', 'glwe@qw.com', 23, 1, 'n')
1:입력|2:전체출력|3:삭제|4:이름찾기|5:내보내기(CSV)|9:종료메뉴 선택 : 3
삭제할 이름은? 강기영
1:입력|2:전체출력|3:삭제|4:이름찾기|5:내보내기(CSV)|9:종료메뉴 선택 : 5
저장할 파일 이름은?member_oracle.csv
1:입력|2:전체출력|3:삭제|4:이름찾기|5:내보내기(CSV)|9:종료메뉴 선택 : 9


In [2]:
#1. 입력
def insert_customer_info(cursor):
    name = input("이름: ")
    phone = input("전화번호: ")
    email = input("이메일: ")
    try:
        age = int(input("나이 : "))
    except ValueError as e:
        print('유효하지 않은 나이  입력시 나이는 0으로 초기화')
        age = 0
    try:
        grade = int(input("고객등급(1~5) : "))
        if grade < 1 :
            grade = 1
        if grade > 5 :
            greade = 5
    except ValueError as e:
        print('유효하지 않은 등급을 입력시 등급은 1로 초기화')
        grade = 1
    etc = input("세부사항: ")
    cursor.execute("INSERT INTO MEMBERS VALUES (:name, :phone, :email, :age, :grade, :etc)", 
                   {'name':name, 'phone':phone, 'email':email, 'age':age, 'grade':grade, 'etc':etc})
    conn.commit()

In [3]:
# 2. 전체 출력
def print_customers(cursor):
    import pandas as pd
    cursor.execute("SELECT * FROM MEMBERS")
    customers = cursor.fetchall()
    data_df = pd.DataFrame(customers)
    data_df.columns = [row[0] for row in cursor.description]
    print(data_df)

In [4]:
# 3. 삭제
def delete_customer(cursor):
    name = input('삭제할 이름은? ')
    cursor.execute("DELETE FROM MEMBERS WHERE NAME=:name", {'name':name})
    conn.commit()

In [5]:
# 4. 이름찾기(동명이인도 모두 검색)
def search_customer(cursor):
    name = input('검색할 이름은? ')
    cursor.execute("SELECT * FROM MEMBERS WHERE NAME=:name", {'name':name})
    customers = cursor.fetchall()
    if len(customers)==0:
        print('검색한 고객이 데이터에 존재하지 않습니다')
    else:
        for c in customers:
            print(c)


In [6]:
def lid(keys, values):
    return{key: value for key, value in zip (keys, values)}

In [7]:
# 5. 내보내기(CSV)
def save_customer_csv(cursor):
    import csv
    customer_dict_list = []
    cursor.execute("SELECT * FROM MEMBER")
    customers = cursor.fetchall()
    fieldname = [row[0] for row in cursor.description]
    for c in customers:
        temp = list(c)
        customer_dict_list.append(lid(fieldname, temp))
    filename = input('저장할 파일 이름은?')
    try:
        with open(filename, "w", newline='', encoding="CP949") as f:
            dict_writer = csv.DictWriter(f, fieldnames=fieldname)
            dict_writer.writeheader()
            dict_writer.writerows(customer_dict_list)
    except Exception as e:
        print('데이터 내보내기 예외 :', e)